In [63]:

import tableauserverclient as TSC
import pandas as pd
import os

print(os.environ.get('http_proxy'))
print(os.environ.get('HTTP_PROXY'))

os.environ['http_proxy'] = ""
os.environ['HTTP_PROXY'] = ""
os.environ['https_proxy'] = ""
os.environ['HTTPS_PROXY'] = ""
print(os.environ.get('http_proxy'))
print(os.environ.get('HTTP_PROXY'))


# Get Site name

site_name=[]
tableau_auth = TSC.TableauAuth('da-datamgmt', 'CJD48')
server = TSC.Server('http://tableau.cjd.th', use_server_version=True)
server.auth.sign_in(tableau_auth)

all_sites, pagination_item = server.sites.get()
for site in all_sites:
    site_name.append(site.content_url.replace(' ',''))
    
# Remove Default site 
site_name=site_name[1:]

server.auth.sign_out()


In [64]:

### sign in

relationship_df = pd.DataFrame(columns=['Site_Name','Project_Name','Dashboard_Name', 'Workbook_Name', 'Description', 'Connection_Type', 'Database_Connection', 'Datasource_Name', 'Table_Name','Owner'])
refresh_df = pd.DataFrame(columns=['Site_Name', 'name', 'hasExtracts', 'extractLastRefreshTime', 'extractLastIncrementalUpdateTime', 'extractLastUpdateTime'])


for site in site_name:
    print(site)
    tableau_auth = TSC.TableauAuth('da-datamgmt', 'CJD48', site)
    
    if site == "":
        temp_site = "Default"
    else:
        temp_site = site

    server = TSC.Server('http://tableau.cjd.th', use_server_version=True)
    server.auth.sign_in(tableau_auth)

# to filter a query (filter: {name: "dwd_cf_lcm_t_application_i_d"})

    query = """
    {
      workbooks {
          name 
          projectName
          upstreamDatabases {
              name 
              connectionType     
          }
          embeddedDatasources {
            name
            upstreamTables{
              name
            }
          }
          description
          owner{
              name
              username
              email
          }
          views{
              name
          }   
          
      }
    }"""
  
    resp = server.metadata.query(query)
    List_Data = resp['data']['workbooks']
   
    for dict in List_Data:
        wbName = dict['name']
        wbProject = dict['projectName']
      
        wbOwner = dict['owner']['username']
        desc = dict['description']
        for view in dict['views']:
            wbView = view['name']
        for datasource in dict['upstreamDatabases']:
            dbCon = datasource['name']
            connectionType = datasource['connectionType']
        for datasourceName in dict['embeddedDatasources']:
            dsName = datasourceName['name']
            for table in datasourceName['upstreamTables']:
                tbName = table['name']
                relationship_df = relationship_df.append({'Site_Name': temp_site,'Project_Name':wbProject,'Dashboard_Name':wbView,
                                                          'Workbook_Name': wbName, 'Description': desc, 'Connection_Type': connectionType, 
                                                          'Database_Connection': dbCon, 'Datasource_Name': dsName, 'Table_Name': tbName, 'Owner': wbOwner}, ignore_index=True)

    query = """
    {
      publishedDatasources {
        name
        hasExtracts
        extractLastRefreshTime
        extractLastIncrementalUpdateTime
        extractLastUpdateTime 
        downstreamWorkbooks{
        name
        }
      }
    }"""

    resp = server.metadata.query(query)

    refresh_df2 = pd.DataFrame(resp['data']['publishedDatasources'])
    refresh_df2['Site_Name'] = site
    ## Re-order the data frame
    try:
        
        refresh_df2 =  refresh_df2[['Site_Name', 'name', 'hasExtracts', 'extractLastRefreshTime', 'extractLastIncrementalUpdateTime', 'extractLastUpdateTime']]
        refresh_df=refresh_df.append(refresh_df2)
        del [[refresh_df2]]
        server.auth.sign_out()
    except:
        pass

refresh_df=refresh_df.reset_index(drop=True)


financeandaccounting
DataAnalyticDepartment
DolfinWallet
MarketingDepartment
ConsumerFinanceDepartment
MerchantDepartment
ProductDepartment
StrategyDepartment
ComplianceDepartment
OperationDepartment
ITDepartment
FraudDepartment
DolfinExplorer


In [65]:
group_site_table = relationship_df.groupby(['Site_Name', 'Datasource_Name', 'Table_Name'])
count_site_table = group_site_table['Table_Name'].count()
group_table = count_site_table.groupby(['Table_Name'])
chk_dup_table = group_table.count()

duplicates = chk_dup_table[chk_dup_table > 1].index.tolist()

def chk_dup_fn(row):
    if row['Table_Name'] in duplicates:
        return "Y"
    return "N"

relationship_df['Flag_Duplicate'] = relationship_df.apply(lambda row: chk_dup_fn(row), axis=1)
refresh_df = refresh_df[refresh_df['hasExtracts']==True]
        
#join both table
final=relationship_df.merge(refresh_df.rename({'name': 'Datasource_Name'}, axis=1),
               left_on=['Site_Name','Datasource_Name'], right_on=['Site_Name','Datasource_Name'], how='left')
final = final.drop_duplicates()
final=final.reset_index(drop=True)

final.to_excel("wb_datasources_refresh.xlsx", encoding='utf-8', index=False)

In [66]:
relationship_df

,Site_Name,Project_Name,Dashboard_Name,Workbook_Name,Description,Connection_Type,Database_Connection,Datasource_Name,Table_Name,Owner,Flag_Duplicate
0,financeandaccounting,PRODUCTION,Sheet 1 (2),finance_wl_opersupport_d_T1CC Point Issue Reco...,,hadoophive,10.223.35.31:10000,app_wl_operreport_t1cc_point_a_m,app_wl_operreport_t1cc_point_a_m,walaya.ma,N
1,financeandaccounting,OUT OF DATE,Merchant Organic Expenses,dolfin_organic_payment_analysis_sandbox,,hadoophive,10.223.35.31:10000,Adjusted Dws.Transaction,da_dev.benz_wl_txn_and_ecoupon_v3,vorawat.ch,N
2,financeandaccounting,PRODUCTION,ENDING_BALANCE,finance_daily_wallet_balance_by_channel,,hadoophive,10.223.35.31:10000,app_wl_finance_balance_by_channel_i_d,app_wl_finance_balance_by_channel_i_d,tableau-admins,Y
3,financeandaccounting,PRODUCTION,ENDING_BALANCE,finance_daily_wallet_balance_by_channel,,hadoophive,10.223.35.31:10000,app_wl_finance_customer_daily_movement_i_d,app_wl_finance_balance_by_channel_i_d,tableau-admins,Y
4,financeandaccounting,PRODUCTION,ENDING_BALANCE,finance_daily_wallet_balance_by_channel,,hadoophive,10.223.35.31:10000,app_wl_finance_customer_daily_movement_i_d,app_wl_finance_customer_daily_movement_i_d,tableau-admins,N
...,...,...,...,...,...,...,...,...,...,...,...
460,DolfinExplorer,MERCHANT,Total Tran by Merchant,dolfin_explorer_merchant_transaction_by_channel,,hadoophive,10.223.35.31:10000,dws_wl_transaction_a_d,dws.dws_wl_transaction_a_d,tableau-admins,N
461,DolfinExplorer,MERCHANT,Total Tran by Merchant,dolfin_explorer_merchant_transaction_by_channel,,hadoophive,10.223.35.31:10000,dws_wl_transaction_a_d,dwd.dwd_merchant_merchant_library_a_d,tableau-admins,Y
462,DolfinExplorer,FINANCE AND ACCOUNTING,cohort_test,FINANCE_POC,,hadoophive,10.223.35.31:10000,dws_wl_transaction_a_d,dws.dws_wl_transaction_a_d,narudh.ar,N
463,DolfinExplorer,FINANCE AND ACCOUNTING,cohort_test,FINANCE_POC,,hadoophive,10.223.35.31:10000,dws_wl_transaction_a_d,dwd.dwd_merchant_merchant_library_a_d,narudh.ar,Y


In [67]:
refresh_df

,Site_Name,name,hasExtracts,extractLastRefreshTime,extractLastIncrementalUpdateTime,extractLastUpdateTime
0,financeandaccounting,app_wl_promo_txn_i_d,True,2020-05-12T02:43:17Z,None,2020-05-12T02:43:17Z
1,financeandaccounting,app_wl_point_reconcile_i_d,True,2020-05-12T05:01:47Z,None,2020-05-12T05:01:47Z
4,financeandaccounting,app_wl_finance_balance_by_channel_i_d,True,2020-02-25T00:04:05Z,None,2020-02-25T00:04:05Z
5,financeandaccounting,app_wl_campaign_campaignmonitoring_a_d,True,2020-05-12T01:01:03Z,None,2020-05-12T01:01:03Z
6,financeandaccounting,app_wl_coupon_master_i_d,True,2020-05-12T02:01:16Z,None,2020-05-12T02:01:16Z
...,...,...,...,...,...,...
247,FraudDepartment,app_wl_antifraud_dashboard_c_a_d,True,2020-05-12T03:01:45Z,None,2020-05-12T03:01:45Z
248,FraudDepartment,t1_tws_recencycust,True,2020-04-07T07:12:49Z,None,2020-04-07T07:12:49Z
249,DolfinExplorer,Transaction (Sample Super Wallet),True,2019-12-18T09:30:04Z,None,2019-12-18T09:30:04Z
252,DolfinExplorer,dwd_merchant_merchant_library_a_d,True,2020-05-12T04:01:53Z,None,2020-05-12T04:01:53Z


In [68]:
final

,Site_Name,Project_Name,Dashboard_Name,Workbook_Name,Description,Connection_Type,Database_Connection,Datasource_Name,Table_Name,Owner,Flag_Duplicate,hasExtracts,extractLastRefreshTime,extractLastIncrementalUpdateTime,extractLastUpdateTime
0,financeandaccounting,PRODUCTION,Sheet 1 (2),finance_wl_opersupport_d_T1CC Point Issue Reco...,,hadoophive,10.223.35.31:10000,app_wl_operreport_t1cc_point_a_m,app_wl_operreport_t1cc_point_a_m,walaya.ma,N,True,2020-05-12T05:01:32Z,None,2020-05-12T05:01:32Z
1,financeandaccounting,OUT OF DATE,Merchant Organic Expenses,dolfin_organic_payment_analysis_sandbox,,hadoophive,10.223.35.31:10000,Adjusted Dws.Transaction,da_dev.benz_wl_txn_and_ecoupon_v3,vorawat.ch,N,NaN,NaN,NaN,NaN
2,financeandaccounting,PRODUCTION,ENDING_BALANCE,finance_daily_wallet_balance_by_channel,,hadoophive,10.223.35.31:10000,app_wl_finance_balance_by_channel_i_d,app_wl_finance_balance_by_channel_i_d,tableau-admins,Y,True,2020-02-25T00:04:05Z,None,2020-02-25T00:04:05Z
3,financeandaccounting,PRODUCTION,ENDING_BALANCE,finance_daily_wallet_balance_by_channel,,hadoophive,10.223.35.31:10000,app_wl_finance_customer_daily_movement_i_d,app_wl_finance_balance_by_channel_i_d,tableau-admins,Y,True,2020-02-25T00:10:19Z,None,2020-02-25T00:10:19Z
4,financeandaccounting,PRODUCTION,ENDING_BALANCE,finance_daily_wallet_balance_by_channel,,hadoophive,10.223.35.31:10000,app_wl_finance_customer_daily_movement_i_d,app_wl_finance_customer_daily_movement_i_d,tableau-admins,N,True,2020-02-25T00:10:19Z,None,2020-02-25T00:10:19Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,DolfinExplorer,MERCHANT,Total Tran by Merchant,dolfin_explorer_merchant_transaction_by_channel,,hadoophive,10.223.35.31:10000,dws_wl_transaction_a_d,dws.dws_wl_transaction_a_d,tableau-admins,N,True,2020-05-12T04:06:35Z,None,2020-05-12T04:06:35Z
462,DolfinExplorer,MERCHANT,Total Tran by Merchant,dolfin_explorer_merchant_transaction_by_channel,,hadoophive,10.223.35.31:10000,dws_wl_transaction_a_d,dwd.dwd_merchant_merchant_library_a_d,tableau-admins,Y,True,2020-05-12T04:06:35Z,None,2020-05-12T04:06:35Z
463,DolfinExplorer,FINANCE AND ACCOUNTING,cohort_test,FINANCE_POC,,hadoophive,10.223.35.31:10000,dws_wl_transaction_a_d,dws.dws_wl_transaction_a_d,narudh.ar,N,True,2020-05-12T04:06:35Z,None,2020-05-12T04:06:35Z
464,DolfinExplorer,FINANCE AND ACCOUNTING,cohort_test,FINANCE_POC,,hadoophive,10.223.35.31:10000,dws_wl_transaction_a_d,dwd.dwd_merchant_merchant_library_a_d,narudh.ar,Y,True,2020-05-12T04:06:35Z,None,2020-05-12T04:06:35Z
